In [6]:
#librerias requeridas para trabajar con grafos
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import collections


In [15]:
dt= pd.read_csv("dating.edges"," ")
print(dt)
red = nx.from_pandas_edgelist(dt,source="n",target = "t")

               n      t   w
0              1      1   8
1              2      2   6
2              2      3  10
3              2      4   7
4              2      5  10
...          ...    ...  ..
17359341  135360   8167   2
17359342  135360  11354   2
17359343  135360  32204   5
17359344  135360  40880   5
17359345  135360  61270   8

[17359346 rows x 3 columns]


In [ ]:
nx.draw(red,node_size=5,edge_color='b',node_color="black") 